# llamaindex 加载现有向量数据库


In [1]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
    Settings,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [2]:
# 设置嵌入模型和语言模型
Settings.embed_model = OllamaEmbedding(model_name="yxl/m3e:latest")
Settings.llm = Ollama(model="qwen2.5:0.5b", request_timeout=360)

In [3]:
# 初始化 Chroma 客户端，指定数据存储路径为当前目录下的 chroma_db 文件夹
db = chromadb.PersistentClient(path="./chroma_db")

# 获取或创建名为 "quickstart" 的集合，如果该集合不存在，则创建它
chroma_collection = db.get_or_create_collection("quickstart")

# 使用上述集合创建一个 ChromaVectorStore 实例，以便 llama_index 可以与 Chroma 集合进行交互
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 创建一个存储上下文，指定向量存储为刚刚创建的 ChromaVectorStore 实例
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# 从向量存储创建索引
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # 返回最相似的前 n 个文档片段
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [9]:
# 执行查询
response = query_engine.query("糖尿病如何诊断")
print(response)  # 输出查询结果

糖尿病的诊断通常遵循以下步骤：

1. **确诊糖尿病时或确诊之后出现的神经病变**：需要在明确的糖尿病病史基础上，确认是否有神经病变。
2. **出现神经病变的临床症状**：若确诊，则需要检查神经病变的具体情况，包括疼痛、麻木、感觉异常等。

根据提供的信息和规则：
- **无临床症状则**：无需进行检查
- **出现任何异常则**：需进行相应的检查

因此，糖尿病的诊断主要是确认是否有神经病变。
